In [1]:
sc

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from pyspark.context import SparkContext
from pyspark.conf import SparkConf

import argparse
import os
import numpy
import sys
import tensorflow as tf
import threading
from datetime import datetime

from com.yahoo.ml.tf import TFCluster
import mnist_dist

In [3]:
parser = argparse.ArgumentParser()
parser.add_argument("-i", "--images", help="HDFS path to MNIST images in parallelized format")
parser.add_argument("-f", "--format", help="example format: (csv|pickle|tfr)", choices=["csv","pickle","tfr"], default="csv")
parser.add_argument("-l", "--labels", help="HDFS path to MNIST labels in parallelized format")
parser.add_argument("-m", "--model", help="HDFS path to save/load model during train/test", default="mnist_model")
parser.add_argument("-o", "--output", help="HDFS path to save test/inference output", default="predictions")
parser.add_argument("-r", "--readers", help="number of reader/enqueue threads", type=int, default=1)
parser.add_argument("-s", "--steps", help="maximum number of steps", type=int, default=1000)
parser.add_argument("-X", "--mode", help="train|test", default="train")
parser.add_argument("-tb", "--tensorboard", help="launch tensorboard process", action="store_true")
args = parser.parse_args(['-f', 'csv', '-m', 'mnist_model', '-r', '1', '-i', 'mnist/csv/train/images', '-l', 'mnist/csv/train/labels'])

In [4]:
print(args)

Namespace(format='csv', images='mnist/csv/train/images', labels='mnist/csv/train/labels', mode='train', model='temp_model', output='predictions', readers=1, steps=1000, tensorboard=False)


In [5]:
images = sc.textFile(args.images).map(lambda ln: [int(x) for x in ln.split(',')])
labels = sc.textFile(args.labels).map(lambda ln: [float(x) for x in ln.split(',')])

dataRDD = images.zip(labels)
dataRDD.count()

60000

In [6]:
num_executors = int(sc._conf.get("spark.executor.instances"))
cluster = TFCluster.reserve(sc, num_executors, 1, True, TFCluster.InputMode.SPARK)

In [7]:
for node in cluster.cluster_info:
  print(node)

{'addr': ('gpbl191n10.blue.ygrid.yahoo.com', 38252), 'task_index': 0, 'port': 32974, 'authkey': UUID('dcafb079-8675-4984-8bbc-7e4f0e47c996'), 'worker_num': 0, 'host': 'gpbl191n10.blue.ygrid.yahoo.com', 'ppid': 122387, 'job_name': 'ps', 'tb_port': 0}
{'addr': '/tmp/pymp-JxxA_1/listener-IJlOe1', 'task_index': 0, 'port': 47690, 'authkey': UUID('9cf395ef-9c23-4467-a236-06e70522a465'), 'worker_num': 1, 'host': 'gpbl191n10.blue.ygrid.yahoo.com', 'ppid': 122261, 'job_name': 'worker', 'tb_port': 35200}
{'addr': '/tmp/pymp-sUbkvF/listener-S9WrOv', 'task_index': 1, 'port': 51928, 'authkey': UUID('af3fc0a7-358a-43e9-9b12-ca859feda768'), 'worker_num': 2, 'host': 'gpbl191n10.blue.ygrid.yahoo.com', 'ppid': 122265, 'job_name': 'worker', 'tb_port': 0}
{'addr': '/tmp/pymp-dLO3XF/listener-qmKXZe', 'task_index': 2, 'port': 45604, 'authkey': UUID('5f448d57-954a-4c7b-9044-0975d8e6842d'), 'worker_num': 3, 'host': 'gpbl191n15.blue.ygrid.yahoo.com', 'ppid': 143944, 'job_name': 'worker', 'tb_port': 0}


In [8]:
cluster.start(mnist_dist.map_fun, args)

In [ ]:
cluster.train(dataRDD)

In [13]:
cluster.shutdown()

connecting to ('gpbl191n18.blue.ygrid.yahoo.com', 51157), c4babd7c-aa69-4232-9c85-7be46c83c4be


In [14]:

# cluster = TFCluster.reserve(sc, num_executors, num_ps, True, TFCluster.InputMode.SPARK)
# args.mode = "test"
# cluster.start(mnist_dist.map_fun, args)
# resultRDD = cluster.test(dataRDD)

In [15]:
# resultRDD.first()